In [43]:
#!pip install geopandas rasterio rioxarray rasterstats shapely pyproj matplotlib pandas numpy
!pip install mapclassify

   ---------------------------------------- 0.0/882.2 kB ? eta -:--:--
   ---------------------------------------- 10.2/882.2 kB ? eta -:--:--
   - ------------------------------------- 30.7/882.2 kB 330.3 kB/s eta 0:00:03
   - ------------------------------------- 41.0/882.2 kB 330.3 kB/s eta 0:00:03
   ---- ---------------------------------- 92.2/882.2 kB 525.1 kB/s eta 0:00:02
   ------- ------------------------------ 174.1/882.2 kB 876.1 kB/s eta 0:00:01
   --------------- ------------------------ 337.9/882.2 kB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 553.0/882.2 kB 2.0 MB/s eta 0:00:01
   ------------------------- -------------- 553.0/882.2 kB 2.0 MB/s eta 0:00:01
   ------------------------------------ --- 798.7/882.2 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 849.9/882.2 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 882.2/882.2 kB 1.9 MB/s eta 0:00:00


In [21]:
# Import libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import mapping

In [51]:
raster_path = "tmin_raster.tif"       # GeoTIFF
districts_path = "DISTRITOS.shp"      # Shapefile de distritos

# cargar distritos
gdf = gpd.read_file(districts_path)
print("Original CRS distritos:", gdf.crs)
print("Número de distritos:", len(gdf))

# inspeccionar raster
with rasterio.open(raster_path) as src:
    print("Raster CRS:", src.crs)
    print("Band count:", src.count)
    print("Bounds:", src.bounds)
    print("Dtype:", src.dtypes)
    # stats del band1 (si hay más bandas puedes cambiar el índice)
    band1 = src.read(1)
    print("Band1 raw stats:", np.nanmin(band1), np.nanmax(band1))

Original CRS distritos: EPSG:4326
Número de distritos: 1873
Raster CRS: EPSG:4326
Band count: 5
Bounds: BoundingBox(left=-81.38, bottom=-18.65000000000002, right=-67.1300000000008, top=1.2000000000000002)
Dtype: ('float32', 'float32', 'float32', 'float32', 'float32')
Band1 raw stats: -9.052621 24.636774


In [53]:
with rasterio.open(raster_path) as src:
    if gdf.crs != src.crs:
        gdf = gdf.to_crs(src.crs)

In [55]:
def compute_stats_for_geom(src, geom, band=1, scale_factor=1.0):
    try:
        out_image, _ = mask(src, [mapping(geom)], crop=True)
    except ValueError:
        return dict(count=0, mean=np.nan, minimum=np.nan, maximum=np.nan,
                    std=np.nan, p10=np.nan, p90=np.nan, frost_freq=np.nan)
    arr = out_image[band-1].astype('float32')
    nod = src.nodata
    if nod is not None:
        arr[arr == nod] = np.nan
    data = arr.flatten()
    data = data[~np.isnan(data)]
    if data.size == 0:
        return dict(count=0, mean=np.nan, minimum=np.nan, maximum=np.nan,
                    std=np.nan, p10=np.nan, p90=np.nan, frost_freq=np.nan)
    data = data * scale_factor
    return dict(
        count=int(data.size),
        mean=float(data.mean()),
        minimum=float(data.min()),
        maximum=float(data.max()),
        std=float(data.std(ddof=0)),
        p10=float(np.percentile(data, 10)),
        p90=float(np.percentile(data, 90)),
        frost_freq=float((data <= 0).sum()/data.size)
    )

In [57]:
band = 1       # e.g. Band1 = year 2020
scale_factor = 1  # adjust if your raster is scaled ×10
stats_list = []
with rasterio.open(raster_path) as src:
    gdf['geometry'] = gdf['geometry'].buffer(0)  # fix invalid
    for geom in gdf['geometry']:
        stats_list.append(compute_stats_for_geom(src, geom, band, scale_factor))

stats_df = pd.DataFrame(stats_list)
results = pd.concat([gdf.reset_index(drop=True), stats_df], axis=1)
results.head()

,IDDPTO,DEPARTAMEN,IDPROV,PROVINCIA,IDDIST,DISTRITO,CAPITAL,CODCCPP,AREA,FUENTE,geometry,count,mean,minimum,maximum,std,p10,p90,frost_freq
0,10,HUANUCO,1009,PUERTO INCA,100902,CODO DEL POZUZO,CODO DEL POZUZO,0001,1,INEI,"POLYGON ((-75.31797 -9.29529, -75.3171 -9.2975...",195,10.388412,0.0,22.604116,9.757681,0.0,22.172694,0.451282
1,10,HUANUCO,1009,PUERTO INCA,100904,TOURNAVISTA,TOURNAVISTA,0001,1,INEI,"POLYGON ((-74.64136 -8.82302, -74.64036 -8.828...",143,8.885084,0.0,22.860834,10.917670,0.0,22.481903,0.601399
2,25,UCAYALI,2503,PADRE ABAD,250305,ALEXANDER VON HUMBOLDT,ALEXANDER VON HUMBOLDT,0001,1,INEI,"POLYGON ((-75.02253 -8.74193, -75.02267 -8.742...",25,5.293528,0.0,22.141968,9.419977,0.0,22.079941,0.760000
3,25,UCAYALI,2503,PADRE ABAD,250302,IRAZOLA,SAN ALEJANDRO,0001,1,INEI,"POLYGON ((-75.13864 -8.56712, -75.13956 -8.569...",136,10.149405,0.0,22.481043,11.088708,0.0,22.393934,0.544118
4,25,UCAYALI,2503,PADRE ABAD,250304,NESHUYA,MONTE ALEGRE,0001,1,INEI,"POLYGON ((-75.01589 -8.44637, -75.01585 -8.446...",45,10.840166,0.0,22.385101,11.084105,0.0,22.274546,0.511111


In [73]:
import matplotlib.pyplot as plt

# Histograma de la temperatura mínima promedio por distrito
plt.figure(figsize=(8,6))
plt.hist(results['mean'], bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Temperatura mínima media (°C)')
plt.ylabel('Número de distritos')
plt.title('Distribución de Tmin por distrito')
plt.tight_layout()

# Guardar PNG
plt.savefig("histograma_tmin.png", dpi=300)
plt.close()

In [75]:
# Top 15 distritos más fríos
top_cold = results.nsmallest(15, 'mean')[['DISTRITO', 'mean']]
top_cold.to_csv("ranking_distritos_mas_frios.csv", index=False)

# Top 15 distritos más cálidos
top_hot = results.nlargest(15, 'mean')[['DISTRITO', 'mean']]
top_hot.to_csv("ranking_distritos_mas_calidos.csv", index=False)

In [77]:
# Crear mapa cloroplético
fig, ax = plt.subplots(1, 1, figsize=(8, 10))
results.plot(column='mean', ax=ax, legend=True, scheme='quantiles', k=6,
             edgecolor='0.2', cmap='coolwarm')
ax.set_axis_off()
ax.set_title("Temperatura mínima media por distrito (°C)", fontsize=14)

# Guardar PNG
plt.tight_layout()
plt.savefig("mapa_tmin_distritos.png", dpi=300)
plt.close()